# Verse mappings



In [1]:
#r "nuget:SIL.Scripture,10.1.0"
//#r "nuget:SIL.Scripture,11.0.1"
#r "nuget:Thot,3.3.5"
#r "../src/ClearBible.Engine/bin/Debug/net6.0/ClearBible.Engine.dll"
#r "../../machine/src/SIL.Machine/bin/Debug/netstandard2.0/Clear.SIL.Machine.dll"
#r "../../machine/src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.Translation.Thot.dll"

using SIL.Machine.Corpora;
using SIL.Machine.Tokenization;
using SIL.Machine.Translation;
using SIL.Machine.Translation.Thot;
using static SIL.Machine.Corpora.TokenProcessors;
using SIL.Machine.Utils;
using SIL.Scripture;

using ClearBible.Engine.Translation;
using ClearBible.Engine.Corpora;
using ClearBible.Engine.Persistence;
using ClearBible.Engine.Tokenization;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages SIL.Scripture, 10.1.0 Thot, 3.3.5

## Helper renderers

In [2]:
public static class VerseMappingRenderers
{
    public static void DisplayVerseMapping(VerseMapping verseMapping)
    {
        Console.WriteLine($"{verseMapping}");
        Console.WriteLine($"  SourceVerses:");
        foreach(var verse in verseMapping.SourceVerses)
        {
            Console.WriteLine($"    {verse}");
        }
        Console.WriteLine($"  TargetVerses:");
        foreach(var verse in verseMapping.TargetVerses)
        {
            Console.WriteLine($"    {verse}");
        }
    }

    public static void SourceNotEqualTarget(IEnumerable<VerseMapping> verseMappings, int take)
    {
        Console.WriteLine();
        Console.WriteLine("Verse mappings where source does not equal target:");
        foreach (var verseMapping in verseMappings
            .Where(vm => vm.SourceVerses.Count() == 1 &&
                vm.TargetVerses.Count() == 1 && 
                !vm.SourceVerses[0].Equals(vm.TargetVerses[0]))
                .Take(take))
        {
            DisplayVerseMapping(verseMapping);
        }
    }
    
    public static void SourceOrTargetFromDifferentBooks(IEnumerable<VerseMapping> verseMappings)
    {
        Console.WriteLine();
        Console.WriteLine("Verse mappings where source or target includes verses from different books:");
        foreach (var verseMapping in verseMappings
            .Where(vm => 
            {
                HashSet<String> books = new HashSet<string>(vm.SourceVerses
                    .Select(v => v.Book)
                    .Distinct());

                books.UnionWith(vm.TargetVerses
                    .Select(v => v.Book)
                    .Distinct());
                return books.Count() > 1;
            }))
        {
            DisplayVerseMapping(verseMapping);
        }    
    }
    
    public static void VerseMappingsWithMoreThanOneVerse(IEnumerable<VerseMapping> verseMappings)
    {
        Console.WriteLine();
        Console.WriteLine("Verse mappings where source or target includes verses from different books:");
        foreach (var verseMapping in verseMappings
            .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1)))
        {
            DisplayVerseMapping(verseMapping);
        }  
    }
}

## Generating verse mappings for entire bible, both English versification

In [3]:
var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, ScrVers.English);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 && vm.TargetVerses.Count() == 1 && vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

Count of versemappings: 38393
Count of versemappings with more than one verse: 0
Count of versemappings with one verse: 38393
Count of versemappings with source verse equalling target verse: 38393


## Generating verse mappings for entire bible, English to Original versification

In [4]:
var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, ScrVers.Original);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 && vm.TargetVerses.Count() == 1 && vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

VerseMappingRenderers.VerseMappingsWithMoreThanOneVerse(verseMappings);
VerseMappingRenderers.SourceNotEqualTarget(verseMappings, 50);
VerseMappingRenderers.SourceOrTargetFromDifferentBooks(verseMappings);


Count of versemappings: 38510
Count of versemappings with more than one verse: 3
Count of versemappings with one verse: 38507
Count of versemappings with source verse equalling target verse: 35740

Verse mappings where source or target includes verses from different books:
1SA 20:42
  SourceVerses:
    1SA 20:42
  TargetVerses:
    1SA 20:42
    1SA 21:1
1KI 22:43
  SourceVerses:
    1KI 22:43
  TargetVerses:
    1KI 22:43
    1KI 22:44
1CH 12:4
  SourceVerses:
    1CH 12:4
  TargetVerses:
    1CH 12:4
    1CH 12:5

Verse mappings where source does not equal target:
GEN 31:55
  SourceVerses:
    GEN 31:55
  TargetVerses:
    GEN 32:1
GEN 32:1
  SourceVerses:
    GEN 32:1
  TargetVerses:
    GEN 32:2
GEN 32:2
  SourceVerses:
    GEN 32:2
  TargetVerses:
    GEN 32:3
GEN 32:3
  SourceVerses:
    GEN 32:3
  TargetVerses:
    GEN 32:4
GEN 32:4
  SourceVerses:
    GEN 32:4
  TargetVerses:
    GEN 32:5
GEN 32:5
  SourceVerses:
    GEN 32:5
  TargetVerses:
    GEN 32:6
GEN 32:6
  SourceVerses

## ## Generating verse mappings for entire bible, CustomEnglish to English versification

In [5]:
using System.IO;

Versification.Table.Implementation.RemoveAllUnknownVersifications();
string customVersificationAddition = "&MAT 1:2 = MAT 1:1\nMAT 1:3 = MAT 1:2\nMAT 1:1 = MAT 1:3\n";
ScrVers customEnglishVersification;
using (var reader = new StringReader(customVersificationAddition))
{
    customEnglishVersification = Versification.Table.Implementation.Load(reader, "vers.txt", ScrVers.English, "custom");
}

var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.English, customEnglishVersification);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 &&
        vm.TargetVerses.Count() == 1 && 
        vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");


VerseMappingRenderers.VerseMappingsWithMoreThanOneVerse(verseMappings);
VerseMappingRenderers.SourceNotEqualTarget(verseMappings, 50);
VerseMappingRenderers.SourceOrTargetFromDifferentBooks(verseMappings);



Count of versemappings: 38379
Count of versemappings with more than one verse: 11
Count of versemappings with one verse: 38368
Count of versemappings with source verse equalling target verse: 38365

Verse mappings where source or target includes verses from different books:
ESG 4:21
  SourceVerses:
    ESG 4:21
  TargetVerses:
    ESG 4:20
    ESG 4:21
ESG 4:23
  SourceVerses:
    ESG 4:23
  TargetVerses:
    ESG 4:22
    ESG 4:23
ESG 4:31
  SourceVerses:
    ESG 4:31
  TargetVerses:
    ESG 4:29
    ESG 4:30
    ESG 4:31
ESG 4:35
  SourceVerses:
    ESG 4:35
  TargetVerses:
    ESG 4:34
    ESG 4:35
ESG 4:37
  SourceVerses:
    ESG 4:37
  TargetVerses:
    ESG 4:36
    ESG 4:37
ESG 5:4
  SourceVerses:
    ESG 5:4
  TargetVerses:
    ESG 5:2
    ESG 5:3
    ESG 5:4
ESG 5:10
  SourceVerses:
    ESG 5:10
  TargetVerses:
    ESG 5:9
    ESG 5:10
ESG 5:12
  SourceVerses:
    ESG 5:12
  TargetVerses:
    ESG 5:11
    ESG 5:12
ESG 5:14
  SourceVerses:
    ESG 5:14
  TargetVerses:
    ESG 5:1

## ## Generating verse mappings for entire bible, RussianOrthodox to English versification

In [6]:
var verseMappings = EngineParallelTextCorpus.VerseMappingsForAllVerses(ScrVers.RussianOrthodox, customEnglishVersification);

var verseMappingsWithMoreThanOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() > 1) || (vm.TargetVerses.Count() > 1));
    
var verseMappingsWithOneVerse = verseMappings
    .Where(vm => (vm.SourceVerses.Count() == 1) && (vm.TargetVerses.Count() == 1));

var verseMappingsWithSourceTargetEqual = verseMappings
    .Where(vm => vm.SourceVerses.Count() == 1 &&
        vm.TargetVerses.Count() == 1 && 
        vm.SourceVerses[0].Equals(vm.TargetVerses[0]));
    
Console.WriteLine($"Count of versemappings: {verseMappings.Count()}");
Console.WriteLine($"Count of versemappings with more than one verse: { verseMappingsWithMoreThanOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with one verse: { verseMappingsWithOneVerse.Count()}");
Console.WriteLine($"Count of versemappings with source verse equalling target verse: { verseMappingsWithSourceTargetEqual.Count()}");

VerseMappingRenderers.VerseMappingsWithMoreThanOneVerse(verseMappings);
VerseMappingRenderers.SourceNotEqualTarget(verseMappings, 50);
VerseMappingRenderers.SourceOrTargetFromDifferentBooks(verseMappings);

Count of versemappings: 36891
Count of versemappings with more than one verse: 14
Count of versemappings with one verse: 36877
Count of versemappings with source verse equalling target verse: 34020

Verse mappings where source or target includes verses from different books:
LEV 14:55
  SourceVerses:
    LEV 14:55
  TargetVerses:
    LEV 14:55
    LEV 14:56
NEH 7:69
  SourceVerses:
    NEH 7:69
  TargetVerses:
    NEH 7:68
    NEH 7:69
PSA 12:6
  SourceVerses:
    PSA 12:6
  TargetVerses:
    PSA 13:5
    PSA 13:6
PSA 86:2
  SourceVerses:
    PSA 86:2
  TargetVerses:
    PSA 87:2
    PSA 87:1
PSA 114:8
  SourceVerses:
    PSA 114:8
  TargetVerses:
    PSA 116:8
    PSA 116:9
SNG 1:1
  SourceVerses:
    SNG 1:1
  TargetVerses:
    SNG 1:1
    SNG 1:2
ISA 64:2
  SourceVerses:
    ISA 64:2
  TargetVerses:
    ISA 64:2
    ISA 64:1
ACT 19:40
  SourceVerses:
    ACT 19:40
  TargetVerses:
    ACT 19:40
    ACT 19:41
2CO 11:32
  SourceVerses:
    2CO 11:32
  TargetVerses:
    2CO 11:32
    2CO

    BAR 6:46
LJE 1:47
  SourceVerses:
    LJE 1:47
  TargetVerses:
    BAR 6:47
LJE 1:48
  SourceVerses:
    LJE 1:48
  TargetVerses:
    BAR 6:48
LJE 1:49
  SourceVerses:
    LJE 1:49
  TargetVerses:
    BAR 6:49
LJE 1:50
  SourceVerses:
    LJE 1:50
  TargetVerses:
    BAR 6:50
LJE 1:51
  SourceVerses:
    LJE 1:51
  TargetVerses:
    BAR 6:51
LJE 1:52
  SourceVerses:
    LJE 1:52
  TargetVerses:
    BAR 6:52
LJE 1:53
  SourceVerses:
    LJE 1:53
  TargetVerses:
    BAR 6:53
LJE 1:54
  SourceVerses:
    LJE 1:54
  TargetVerses:
    BAR 6:54
LJE 1:55
  SourceVerses:
    LJE 1:55
  TargetVerses:
    BAR 6:55
LJE 1:56
  SourceVerses:
    LJE 1:56
  TargetVerses:
    BAR 6:56
LJE 1:57
  SourceVerses:
    LJE 1:57
  TargetVerses:
    BAR 6:57
LJE 1:58
  SourceVerses:
    LJE 1:58
  TargetVerses:
    BAR 6:58
LJE 1:59
  SourceVerses:
    LJE 1:59
  TargetVerses:
    BAR 6:59
LJE 1:60
  SourceVerses:
    LJE 1:60
  TargetVerses:
    BAR 6:60
LJE 1:61
  SourceVerses:
    LJE 1:61
  TargetVer